In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import *
import math
from scipy import stats

In [2]:
# Importing data from csv files

rail_dist = pd.read_csv('Rail_dist_in_mm.csv')
trig_angle = pd.read_csv('angle_trig_inmeter.csv')
vink_angle = pd.read_csv('Angle_gen.csv')
string_laser = pd.read_csv('L_string_laser.csv') 
string_tape = pd.read_csv('L_string_tape.csv')
dia = pd.read_csv('ball_radius.csv')

a_df = pd.read_csv('acceleration_final.csv')

rail_dist = rail_dist.to_numpy()
trig_angle = trig_angle.to_numpy()
vink_angle = vink_angle.to_numpy()
string_laser = string_laser.to_numpy()
string_tape = string_tape.to_numpy()
dia = dia.to_numpy()

a_df = a_df.to_numpy()

In [3]:
def error_probagation(f, PAR,SIGMA):
    "GENERAL ERROR PROBAGATION"
    led = 0
    for i in range(len(PAR)):
        led += ((f.diff(PAR[i]) * SIGMA[i])**2)
    return sqrt(led)

In [4]:
# Rail width

mean_rail = np.mean(rail_dist)
error_rail = np.std(rail_dist)/np.sqrt(len(rail_dist))

print('Rail width = ', mean_rail, 'mm' , '±', round(error_rail, 2), 'mm')

Rail width =  10.05 mm ± 0.02 mm


In [5]:
# Trigonometric angle
from sympy.stats import Arcsin, density


hyp_mean = np.mean(trig_angle[:, 0])
kat_mean = np.mean(trig_angle[:, 1])
hyp_error = np.std(trig_angle[:, 0])/np.sqrt(len(trig_angle[:, 0]))
kat_error = np.std(trig_angle[:, 1])/np.sqrt(len(trig_angle[:, 1]))
angle_t = np.arcsin(kat_mean/hyp_mean)
print(hyp_error, kat_error)

# Define symbolic variables
a, b, da, db = symbols('a, b, sigma_a, sigma_b')

# Create the expression arcsin(a/b)
theta = asin(a/b)

par = [a, b]
sig = [da, db]

sig_theta = error_probagation(theta, par, sig)
ftheta = lambdify((a, b), theta)
fsig_theta = lambdify((a, b, da, db), sig_theta)

print("Angle theta:", ftheta(kat_mean, hyp_mean)*(180/np.pi))
print("Error on angle theta:", fsig_theta(kat_mean, hyp_mean, kat_error, hyp_error)*(180/np.pi))

0.004463392767839258 0.004063615231539522
Angle theta: 14.912937331336302
Error on angle theta: 0.2653079841934492


In [6]:
# Vinkelmåler angle

vink_norm = (vink_angle[:, :2])
vink_rev = (vink_angle[:, 3:5])

mean_norm = np.mean(vink_norm, axis=1)[:, np.newaxis]
mean_rev = np.mean(vink_rev, axis=1)[:, np.newaxis]


error_norm = np.std(mean_norm)/np.sqrt(len(mean_norm)) 
error_rev = np.std(mean_rev)/np.sqrt(len(mean_rev))
MEAN_norm = np.mean(mean_norm)  
MEAN_rev = np.mean(mean_rev) 

print(error_norm, error_rev)
array_MEAN = np.hstack((MEAN_norm, MEAN_rev))


# Putting the rev and normal ways of measuring together
MEAN_ang = (MEAN_norm/error_norm**2 + MEAN_rev/error_rev**2)/(1/error_norm**2 + 1/error_rev**2)
error_ang = np.sqrt(1/(1/error_norm**2 + 1/error_rev**2))
chi2_ang = np.sum((array_MEAN - MEAN_ang)**2/error_ang**2)
N_freedom = len(array_MEAN) - 1
chi2_prob = 1 - stats.chi2.cdf(chi2_ang, N_freedom) 


print('Angle = ', MEAN_ang, '±', round(error_ang, 2))
print('Chi2 = ', chi2_ang)
print('P-value = ', chi2_prob)

0.12421629321469846 0.08100925873009844
Angle =  13.711190053285971 ± 0.07
Chi2 =  0.17753863152382643
P-value =  0.6734970906060582


In [7]:
# Chi2 test for the two WAYS (vinkelmåler and trig.) of measuring the angle together

angle_ARRAY = np.hstack((MEAN_ang, ftheta(kat_mean, hyp_mean)*(180/np.pi)))  # [mean angle of vinkelmåler, mean angle of trigonometric angle]
error_ang_array = np.hstack((error_ang, fsig_theta(kat_mean, hyp_mean, kat_error, hyp_error)*(180/np.pi))) # [error on mean angle of vinkelmåler, error on mean angle of trigonometric angle]

WMEAN_ANGTOT = np.sum(angle_ARRAY/error_ang_array**2)/np.sum(1/error_ang_array**2)  # Weighted mean angle
WERROR_ANGTOT = np.sqrt(1/(np.sum(1/error_ang_array**2)))  # Weighted error on mean angle

chi2_angtot = np.sum((angle_ARRAY - WMEAN_ANGTOT)**2/(1/error_ang_array**2))
N_freedom = len(angle_ARRAY) - 1
chi2_prob = 1 - stats.chi2.cdf(chi2_angtot, N_freedom)

print("The mean angle of the two different ways of measurement: ", WMEAN_ANGTOT, "±", round(WERROR_ANGTOT, 2))
print("The chi2 value for the two different ways of measurement: ", chi2_angtot)
print("The p-value for the two different ways of measurement: ", chi2_prob)

The mean angle of the two different ways of measurement:  13.784972481448756 ± 0.07
The chi2 value for the two different ways of measurement:  0.08958046356371731
The p-value for the two different ways of measurement:  0.7647111882819866


In [8]:
# String length

mid_lod = string_tape[:, -1]/2
length_tape_in = string_tape[:, 0]
length_tape_fin = string_tape[:, 1]


m_lti = np.mean(length_tape_in)/100   # initial length of string tape
m_ltf = np.mean(length_tape_fin)/100  # final length of string tape
er_lti = np.std(length_tape_in)/np.sqrt(len(length_tape_in))/100 
er_ltf = np.std(length_tape_fin)/np.sqrt(len(length_tape_fin))/100


length_with_lod = length_tape_in + mid_lod
mean_length = np.mean(length_with_lod)/100
error_length = np.std(length_with_lod)/np.sqrt(len(length_with_lod))/100


print("Mean length of string initial:", m_lti, "m")
print("Mean length of string final:", m_ltf, "m")
print("Error on length of string initial:", er_lti, "m")
print("Error on length of string final:", er_ltf, "m")
print("\n")

print("Mean length of string with lod:", mean_length, "m")
print("Error on length of string with lod:", error_length, "m")

Mean length of string initial: 1.827475 m
Mean length of string final: 1.824375 m
Error on length of string initial: 0.002557922741210145 m
Error on length of string final: 0.0009416574483324621 m


Mean length of string with lod: 1.8420250000000002 m
Error on length of string with lod: 0.002557739480478834 m


In [9]:
# String length with laser

length_laser_in = string_laser[:, 0]
length_laser_fin = string_laser[:, 1]

# Mean and error on length of string initial with laser (Without lod)
m_lli = np.mean(length_laser_in)
e_lli = np.std(length_laser_in)/np.sqrt(len(length_laser_in))
# Mean and error on length of string final with laser (Without lod)
m_llf = np.mean(length_laser_fin)
e_llf = np.std(length_laser_fin)/np.sqrt(len(length_laser_fin))

# Mean and error on length of string initial with laser (With lod)
length_with_lod_laser = length_laser_in + mid_lod/100
mean_length_laser = np.mean(length_with_lod_laser)
error_length_laser = np.std(length_with_lod_laser)/np.sqrt(len(length_with_lod_laser))

# Without lod
print("Mean length of string initial without lod:", m_lli, "m")
print("Error on length of string initial without lod:", e_lli, "m")
print("Mean length of string final without lod:", m_llf, "m")
print("Error on length of string final without lod:", e_llf, "m")
print("\n")

# With lod
print("Mean length of string initial with laser:", mean_length_laser, "m")
print("Error on length of string initial with laser:", round(error_length_laser, 4), "m")

Mean length of string initial without lod: 1.82425 m
Error on length of string initial without lod: 0.00021650635094608582 m
Mean length of string final without lod: 1.824 m
Error on length of string final without lod: 0.0011726039399558468 m


Mean length of string initial with laser: 1.8388 m
Error on length of string initial with laser: 0.0002 m


In [10]:
# Chi2 test of length of string

length_array = np.hstack((mean_length_laser, mean_length))
error_length_array =  np.hstack((error_length_laser, error_length))
weightedmean_length_tot = np.sum(length_array/error_length_array**2)/np.sum(1/error_length_array**2)

print('Weighted mean length = ', weightedmean_length_tot, 'm')
chi2_length = np.sum((length_array - weightedmean_length_tot)**2/(1/error_length_array**2))
N_freedom = len(length_array) - 1
chi2_prob = 1 - stats.chi2.cdf(chi2_length, N_freedom)
print('Chi2 = ', chi2_length)
print('P-value = ', chi2_prob)

Weighted mean length =  1.8388240061162078 m
Chi2 =  6.703204765716491e-11
P-value =  0.9999934674716289


In [11]:
ball_dia = dia[:, 0]
ball_dia_mean = np.mean(ball_dia)
ball_dia_error = np.std(ball_dia)/np.sqrt(len(ball_dia))

print("Mean ball diameter:", ball_dia_mean, "mm")
print("Error on ball diameter:", round(ball_dia_error, 2), "mm")

Mean ball diameter: 14.875 mm
Error on ball diameter: 0.02 mm


In [12]:
# Calculating the gravitational acceleration PENDELUM

# Define symbolic variables
L, T, dL, dT = symbols('L, T, sigma_L, sigma_T')

# Create the expression arcsin(a/b)
g_pendul = L * (2*np.pi/T)**2

par_pendul = [L, T]
sig_pendul = [dL, dT]

sig_g_pendul = error_probagation(g_pendul, par_pendul, sig_pendul)
f_g_pendul = lambdify((L, T), g_pendul)
fsig_g_pendul = lambdify((L, T, dL, dT), sig_g_pendul)

display(sig_g_pendul)

78.9568352087149*sqrt(L**2*sigma_T**2/T**6 + 0.25*sigma_L**2/T**4)

In [13]:
# Writing out the acceleration data

print("Acceleration data :\n", a_df)

acceleration_normal = a_df[:, 1]
acceleration_reverse = a_df[:, 6]

error_normal = a_df[:, 2]
error_reverse = a_df[:, 7]

# Weighted Mean and weighted error on acceleration normal
wm_acc_normal = np.sum(acceleration_normal/(error_normal)**2)/np.sum(1/error_normal**2)
we_acc_normal = np.sqrt(1/np.sum(1/error_normal**2))

# Weighted Mean and weighted error on acceleration reverse
wm_acc_reverse = np.sum(acceleration_reverse/(error_reverse)**2)/np.sum(1/error_reverse**2)
we_acc_reverse = np.sqrt(1/np.sum(1/error_reverse**2))

# Chi2 test of acceleration and total mean of acceleration
acc = np.hstack((wm_acc_normal, wm_acc_reverse))
err_acc = np.hstack((we_acc_normal, we_acc_reverse))

wm_acc = np.sum(acc/err_acc**2)/np.sum(1/err_acc**2) # Weighted mean of acceleration in TOTAL
wm_err_acc = np.sqrt(1/np.sum(1/err_acc**2)) # Weighted error on weighted mean of acceleration in TOTAL

# Chi2 test of acceleration
chi2_acc = np.sum((acc - wm_acc)**2/err_acc**2)
acc_Ndof = len(acc) - 1
prob_acc = 1 - stats.chi2.cdf(chi2_acc, acc_Ndof)

print("\n")
print("Weighted mean of acceleration normal:", wm_acc_normal, "m/s^2 ±",  we_acc_normal, "m/s^2")
print("Weighted mean of acceleration reverse:", wm_acc_reverse, "m/s^2 ±", we_acc_reverse, "m/s^2")
print("\n")

print("Weighted mean of acceleration in TOTAL:", wm_acc, "m/s^2 ±", wm_err_acc, "m/s^2")
print("Chi2 value of acceleration:", chi2_acc)
print("P-value of acceleration:", prob_acc)


Acceleration data :
 [['Emilie:' 1.468 0.006 4.0 0.133 nan 1.569 0.005 2.6 0.267]
 ['Nete:' 1.46656 0.005 6.1 0.046 nan 1.56798 0.005 1.7 0.436]
 ['Emma' 1.471 0.006 3.155 0.206 nan 1.566 0.005 2.982 0.225]
 ['Ervin' 1.4663 0.0066 2.8899 0.2358 nan 1.568 0.0061 0.9954 0.6079]]


Weighted mean of acceleration normal: 1.4678878320161166 m/s^2 ± 0.0029048125316091544 m/s^2
Weighted mean of acceleration reverse: 1.5677222118129255 m/s^2 ± 0.002609316768222288 m/s^2


Weighted mean of acceleration in TOTAL: 1.523139710026629 m/s^2 ± 0.0019411552246047993 m/s^2
Chi2 value of acceleration: 653.7187620175733
P-value of acceleration: 0.0


In [14]:
# Calculating ∆theta and error on ∆theta

a_norm, a_rev = symbols('a_norm, a_rev')
theta = symbols('theta')
da_norm, da_rev = symbols('sigma_anorm, sigma_arev')
dtheta = symbols('sigma_theta')


delta_theta  = (a_norm - a_rev)*sin(theta) / ((a_norm + a_rev)*cos(theta))

sig_delta_theta = error_probagation(delta_theta, [a_norm, a_rev, theta], [da_norm, da_rev, dtheta])
f_delta_theta = lambdify((a_norm, a_rev, theta), delta_theta)
fsig_delta_theta = lambdify((a_norm, a_rev, theta, da_norm, da_rev, dtheta), sig_delta_theta)

display(sig_delta_theta)
display(delta_theta)

delta_theta_value = f_delta_theta(wm_acc_normal, wm_acc_reverse, WMEAN_ANGTOT)
error_delta_theta = fsig_delta_theta(wm_acc_normal, wm_acc_reverse, WMEAN_ANGTOT, we_acc_normal, we_acc_reverse, WERROR_ANGTOT)

print("∆theta = ", delta_theta_value)
print("Error on ∆theta = ", error_delta_theta)

sqrt(sigma_anorm**2*(-(a_norm - a_rev)*sin(theta)/((a_norm + a_rev)**2*cos(theta)) + sin(theta)/((a_norm + a_rev)*cos(theta)))**2 + sigma_arev**2*(-(a_norm - a_rev)*sin(theta)/((a_norm + a_rev)**2*cos(theta)) - sin(theta)/((a_norm + a_rev)*cos(theta)))**2 + sigma_theta**2*((a_norm - a_rev)*sin(theta)**2/((a_norm + a_rev)*cos(theta)**2) + (a_norm - a_rev)/(a_norm + a_rev))**2)

(a_norm - a_rev)*sin(theta)/((a_norm + a_rev)*cos(theta))

∆theta =  -0.08948624634470091
Error on ∆theta =  0.018505300674510457


In [22]:
# Calculating the gravitational acceleration BALL ON INCLINE

a, theta_ball, del_theta, D_ball, d_rail, = symbols('a, theta_ball, del_theta, D_ball, d_rail')
da, dtheta_ball, d_deltheta, dDball, dd_rail = symbols('sigma_a, sigma_theta_ball, sigma_del_theta, sigma_D_ball, sigma_d_rail')

g_ball = a/(sin(theta_ball + del_theta)) * (1 + 2/5 * (D_ball**2/(D_ball**2 - d_rail**2)))

par_ball = [a, theta_ball, del_theta, D_ball, d_rail]
sig_ball = [da, dtheta_ball, d_deltheta, dDball, dd_rail]

sig_g_ball = error_probagation(g_ball, par_ball, sig_ball)
f_g_ball = lambdify((a, theta_ball, del_theta, D_ball, d_rail), g_ball) 
fsig_g_ball = lambdify((a, theta_ball, del_theta, D_ball, d_rail, da, dtheta_ball, d_deltheta, dDball, dd_rail), sig_g_ball)

display(g_ball) 
display(sig_g_ball)

g_value  = f_g_ball(wm_acc_normal, WMEAN_ANGTOT*np.pi/180, delta_theta_value*np.pi/180, ball_dia_mean, mean_rail)

print(delta_theta_value)

error_g = fsig_g_ball(wm_acc_normal, WMEAN_ANGTOT, delta_theta_value, ball_dia_mean, mean_rail, we_acc_normal, 
                             WERROR_ANGTOT, error_delta_theta, ball_dia_error, error_rail)


print("Gravitational acceleration normal:", g_value, "m/s^2 ±", error_g, "m/s^2")

a*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)/sin(del_theta + theta_ball)

sqrt(0.64*D_ball**4*a**2*d_rail**2*sigma_d_rail**2/((D_ball**2 - d_rail**2)**4*sin(del_theta + theta_ball)**2) + 0.64*a**2*sigma_D_ball**2*(-D_ball**3/(D_ball**2 - d_rail**2)**2 + D_ball/(D_ball**2 - d_rail**2))**2/sin(del_theta + theta_ball)**2 + a**2*sigma_del_theta**2*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)**2*cos(del_theta + theta_ball)**2/sin(del_theta + theta_ball)**4 + a**2*sigma_theta_ball**2*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)**2*cos(del_theta + theta_ball)**2/sin(del_theta + theta_ball)**4 + sigma_a**2*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)**2/sin(del_theta + theta_ball)**2)

-0.08948624634470091
Gravitational acceleration normal: 10.762567752658562 m/s^2 ± 0.0913729608033081 m/s^2


In [20]:
wm_acc_normal*(0.4*ball_dia_mean**2/(ball_dia_mean**2 + mean_rail**2) + 1)/np.sin(delta_theta_value + WMEAN_ANGTOT)

2.0696354470470197